In [1]:
import numpy as np
import pandas as pd

In [2]:
INPUT_PATH_1 = "./Cumulative Tests over Time by County_Legacy.xlsx"
INPUT_PATH_2 = "./Cumulative Tests over Time by County.xlsx"
OUTPUT_PATH = "./time_series_covid19_tests_Texas_counties_transformation.csv"

In [3]:
aggregated_list  = []
for i, input_path in enumerate([INPUT_PATH_1, INPUT_PATH_2]):
    df_raw = pd.read_excel(input_path, header=1, nrows=254)

    if 0 == i:
        df_raw["Tests Through May 5*"].replace("--", np.nan, inplace=True)
        df_raw["Tests Through May 23"].replace("-", np.nan, inplace=True)
        df_raw["Tests Through June 10"].replace("-", np.nan, inplace=True)
        df_raw.rename(columns={"Tests Through May 5*": "Tests Through May 5"}, inplace=True)
        new_column_dict = pd.to_datetime(df_raw.columns[1:].to_series().map(
            lambda column: (column[14:] + ", 2020")
        )).dt.strftime('%Y-%m-%d').to_dict()
        df_raw.rename(columns=new_column_dict, inplace=True)
        col_2020_09_12 = df_raw["2020-09-12"]

    if 1 == i:
        # Help to calculate 2020-09-13 - 2020-09-12
        df_raw.insert(1, "2020-09-12", col_2020_09_12)

    df_locations = df_raw[["County"]]
    df_diff = df_raw.iloc[:, 1:].diff(axis=1).iloc[:, 1:]
    aggregated_list.append(pd.concat([df_locations, df_diff], axis = 1).melt(
        id_vars=["County"], var_name="Date", value_name="Number of Tests"
    ))

In [4]:
df_aggregated = pd.concat(aggregated_list, axis=0)
df_aggregated

,County,Date,Number of Tests
0,Anderson,2020-04-22,12.0
1,Andrews,2020-04-22,0.0
2,Angelina,2020-04-22,29.0
3,Aransas,2020-04-22,1.0
4,Archer,2020-04-22,1.0
...,...,...,...
110231,Wood,2021-11-20,4.0
110232,Yoakum,2021-11-20,1.0
110233,Young,2021-11-20,1.0
110234,Zapata,2021-11-20,3.0


In [5]:
df_aggregated.to_csv(OUTPUT_PATH, index=False)
